## Installing Transformers library

In [1]:
#Installing transformer package 
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.7 MB/s eta 0:00:00


## Rating Estimation by ChatGPT

In [ ]:
!pip install openai

In [ ]:
import os 
import openai
os.environ["OpenAI_API_Key"] = ""
api_key = os.environ.get("OpenAI_API_Key")
openai.api_key = api_key

In [ ]:
# Loss to be added in the custom loss
def get_chatgpt_rating(prompt, sample):
  completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", 
     "content": f"{prompt} {sample}"}
  ])
  return 10 - int(completion.choices[0].message)

In [ ]:
# Different prompts tested in order to generate a sensible rating 
prompt1 = "Provide me rating between 0 and 10 (without any explanation), where 0 is the best and 10 is the worst, for the following story summary: " 
prompt2 = "Assign a rating between 0 (best) and 10 (worst) to the given artificial story summary (only give rating as the response):"
prompt3 = "Assign a rating between 0 (best) and 10 (worst) to the given artificial story summary (only give rating as the response). The rating should be based on writing style, coherence and capture strength. Summary:"
prompt4 = "Assign a rating between 0 and 10 to the given artificial story summary. Only give rating as the response (no reasoning). The rating should be based on writing style, coherence, and capture strength. Summary:" # Best
prompt5 = "Provide me rating between 0 and 10 (without any explanation),  for the following story summary:" 

## Data Tokenization/Encoding

In [1]:
#Loading the standard T5 small model and tokenizer 
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Can try different T5 models such as T5-large, T5-3B, T5-11B
base_tokenizer = AutoTokenizer.from_pretrained('t5-small')
base_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

C:\Users\anshj\anaconda3\envs\torch-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Encoding the sequences
def encode_sequences(x, base_tokenizer = base_tokenizer):
  # try:
    # Input consists of different aspects of the story on which the output will be conditioned
    input = x['Input']
    # Label is the conditioned output - Story
    label = x['Summary']
    # Max length of the input sequence in T5 is 512 tokens (BART could be used for longer sequences - 1024 max length limit) 
    model_input = base_tokenizer(input, max_length = 512, truncation=True, padding='max_length')
    model_input['labels'] = base_tokenizer(label, max_length = 512, truncation=True, padding='max_length')['input_ids']
    return model_input

In [3]:
# Loading the final dataset
import pandas as pd
df = pd.read_csv('Plot_Summary_Dataset')

In [4]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.1, random_state=1)

In [5]:
# Tokenizing the dataset
train_df = train_df.apply(encode_sequences, axis=1)
test_df = test_df.apply(encode_sequences, axis = 1)

In [6]:
train_df.dropna(inplace = True)
test_df.dropna(inplace = True)

In [7]:
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

## Number of parameters

In [5]:
# p is accessing each layer's weight tensors of the model
# p.requires_grad() checks whether the layer is frozen for training or not
# p.numel returns total number of elements in the weight tensor - total number of parameters
print("Parameters:", sum(p.numel() for p in base_model.parameters() if p.requires_grad))

Parameters: 60506624


## Accessing different layers of a Model

In [ ]:
base_model

In [52]:
# We access each layer by the key. That is to access (x), we use base_model.x
# Try and go layer by layer. First check the output of a.b, then go for a.b.c
layer_id = 0
base_model.encoder.block[layer_id].layer[0].SelfAttention.o

Linear(in_features=512, out_features=512, bias=False)

In [40]:
base_model.encoder.block[layer_id].layer[1].DenseReluDense.wo

Linear(in_features=2048, out_features=512, bias=False)

In [37]:
base_model.decoder.block[layer_id].layer[0].SelfAttention.o

Linear(in_features=512, out_features=512, bias=False)

In [46]:
base_model.decoder.block[layer_id].layer[2].DenseReluDense.wo

Linear(in_features=2048, out_features=512, bias=False)

## T5 w Adapters

Could also utilize [adapter-transformers](https://github.com/adapter-hub/adapter-transformers) library - provides easy abstract way of incorporating adapters in the models

In [8]:
# Freeze all layers
for param in base_model.parameters():
    param.requires_grad = False

In [9]:
# Function to countnumber of trainable paramters in a layer/block
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [10]:
import torch
# Creating a custom Adapter layer
def make_adapter(in_dim, bottleneck_dim, out_dim):
    adapter_layers = torch.nn.Sequential(
        torch.nn.Linear(in_dim, bottleneck_dim),
        torch.nn.GELU(),
        torch.nn.Linear(bottleneck_dim, out_dim),
    )
    return adapter_layers

In [11]:
total_size = 0
bottleneck_size = 32 # hyperparameter

for layer_id in range(6):

    #############################################################
    # insert 1st adapter layer into transformer block in Encoder
    #############################################################

    orig_layer_1 = base_model.encoder.block[layer_id].layer[0].SelfAttention.o

    adapter_layers_1 = make_adapter(
        in_dim=orig_layer_1.out_features, 
        bottleneck_dim=bottleneck_size, 
        out_dim=orig_layer_1.out_features)

    new_1 = torch.nn.Sequential(orig_layer_1, *adapter_layers_1)
    base_model.encoder.block[layer_id].layer[0].SelfAttention.o = new_1
    
    total_size += count_parameters(adapter_layers_1)

    #############################################################
    # insert 2nd adapter layer into transformer block in Encoder 
    #############################################################

#     orig_layer_2 = base_model.encoder.block[layer_id].layer[1].DenseReluDense.wo

#     adapter_layers_2 = make_adapter(
#         in_dim=orig_layer_2.out_features, 
#         bottleneck_dim=bottleneck_size, 
#         out_dim=orig_layer_2.out_features)

#     new_2 = torch.nn.Sequential(orig_layer_2, *adapter_layers_2)
#     base_model.encoder.block[layer_id].layer[1].DenseReluDense.wo = new_2
    
#     total_size += count_parameters(adapter_layers_2)

    #############################################################
    # insert 1st adapter layer into transformer block in Decoder
    #############################################################

    orig_layer_1 = base_model.decoder.block[layer_id].layer[0].SelfAttention.o

    adapter_layers_1 = make_adapter(
        in_dim=orig_layer_1.out_features, 
        bottleneck_dim=bottleneck_size, 
        out_dim=orig_layer_1.out_features)

    new_1 = torch.nn.Sequential(orig_layer_1, *adapter_layers_1)
    base_model.decoder.block[layer_id].layer[0].SelfAttention.o = new_1
    
    total_size += count_parameters(adapter_layers_1)

    #############################################################
    # insert 2nd adapter layer into transformer block in Decoder 
    #############################################################

#     orig_layer_2 = base_model.decoder.block[layer_id].layer[2].DenseReluDense.wo

#     adapter_layers_2 = make_adapter(
#         in_dim=orig_layer_2.out_features, 
#         bottleneck_dim=bottleneck_size, 
#         out_dim=orig_layer_2.out_features)

#     new_2 = torch.nn.Sequential(orig_layer_2, *adapter_layers_2)
#     base_model.decoder.block[layer_id].layer[2].DenseReluDense.wo = new_2
    
#     total_size += count_parameters(adapter_layers_2)

print("Number of adapter parameters added:", total_size)

Number of adapter parameters added: 399744


## T5 w LoRA

In [18]:
# Installing PEFT Library
!pip install peft

     ---------------------------------------- 0.0/56.8 kB ? eta -:--:--
     ---------------------------------------- 56.8/56.8 kB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/227.6 kB ? eta -:--:--
     -------------------------------------- 227.6/227.6 kB 7.0 MB/s eta 0:00:00


In [19]:
from peft import get_peft_model, LoraConfig, TaskType

# We are using LoRA for parameter efficient fine tuning of a Seq2SeqLM. Arguments' default values are taken from the demonstrations
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

In [20]:
# Customizing the model foe peft 
base_model = get_peft_model(base_model, peft_config)
# Number of trainable parameters in the model in PEFT model
base_model.print_trainable_parameters()

trainable params: 294912 || all params: 60801536 || trainable%: 0.4850403779272945


## T5 Training Setup (Includes Custom Loss Function)

In [12]:
# Initializing the Data Collator for batching of the dataset
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(
        tokenizer=base_tokenizer,
        return_tensors="pt",
        max_length = 512,
        padding = 'max_length',
        model = base_model
    )

In [ ]:
!pip install accelerate

In [13]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# Path where model training loss and intermediate weights will be stored
"""
Change model path to 
1) /content/drive/MyDrive/Visual Story Telling/Story_Gen_Model/Basic - when using no pEDT methods
2) /content/drive/MyDrive/Visual Story Telling/Story_Gen_Model/Adapters - when using Adapters
3) /content/drive/MyDrive/Visual Story Telling/Story_Gen_Model/LoRA - when using LoRA
"""
model_path = f'Story_Gen_Model/Adapters'


#Specifying the training argument 
training_args = Seq2SeqTrainingArguments(
    output_dir=model_path,
    per_device_train_batch_size=8, 
    overwrite_output_dir = True, 
    evaluation_strategy="no", 
    gradient_accumulation_steps=8, 
    num_train_epochs=15,
    weight_decay=0.01, 
    lr_scheduler_type="cosine",
    learning_rate=5e-4, 
    fp16=True 
)

In [ ]:
# Overwrite the Trainer API for utilizing custom loss function 
import torch
import torch.nn.functional as F
class CustomSeq2SeqTrainer(Seq2SeqTrainer):
  def compute_loss(self, model, inputs, return_outputs=False):
      labels = inputs.pop("labels")
      input_ids = inputs.pop("input_ids")
      attention_mask = inputs.pop("attention_mask")
      outputs = model(input_ids = input_ids, labels = labels, attention_mask = attention_mask)
      loss_cross_entropy = outputs.loss
      logits = outputs.logits
      loss_GPT = custom_loss_function(logits) 
      loss = (loss_cross_entropy + torch.tensor(loss_GPT))
      return (loss, outputs) if return_outputs else loss 

# Check whether the tokenizer is being passed as an argument 
def custom_loss_function(logits, tokenizer=base_tokenizer):
    loss_GPT = 10
    summaries = base_tokenizer.batch_decode(F.softmax(logits, dim=-1).argmax(dim=-1), skip_special_tokens=True)
    for summary in summaries:
      loss_GPT = loss_GPT + (10 - int(get_chatgpt_rating(prompt4, summary)))
    return loss_GPT


In [14]:
# Initializing the trainer

trainer = Seq2SeqTrainer(
    model=base_model,                         # the instantiated  Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    data_collator=data_collator,
    train_dataset=train_df,       # training dataset
    eval_dataset = test_df,
)

In [ ]:
#Custom Trainer function using Customized Loss function 

trainer = CustomSeq2SeqTrainer(
    model=base_model,                         # the instantiated  Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    data_collator=data_collator,
    train_dataset=train_df,       # training dataset
    eval_dataset = test_df,
)

## Model Training 

In [15]:
# Starting the training
trainer.train()

C:\Users\anshj\anaconda3\envs\torch-gpu\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,2.170400
1000,1.796000
1500,1.748000
2000,1.728200
2500,1.700400
3000,1.698600
3500,1.688900
4000,1.674400
4500,1.681500
5000,1.675000


TrainOutput(global_step=8145, training_loss=1.7210480373281873, metrics={'train_runtime': 6142.2723, 'train_samples_per_second': 84.877, 'train_steps_per_second': 1.326, 'total_flos': 7.118346527440896e+16, 'train_loss': 1.7210480373281873, 'epoch': 15.0})

In [16]:
# Saving the final model
trainer.save_model()

## Model Evaluation 

In [17]:
trainer.evaluate()

{'eval_loss': 1.5473278760910034,
 'eval_runtime': 19.6496,
 'eval_samples_per_second': 196.544,
 'eval_steps_per_second': 24.581,
 'epoch': 15.0}

## Loading of the FT model

In [ ]:
"""
Change model path to 
1) /content/drive/MyDrive/Visual Story Telling/Story_Gen_Model/Basic - when using no pEDT methods
2) /content/drive/MyDrive/Visual Story Telling/Story_Gen_Model/Adapters - when using Adapters
3) /content/drive/MyDrive/Visual Story Telling/Story_Gen_Model/LoRA - when using LoRA
"""
model_path = f'/content/drive/MyDrive/Visual Story Telling/Story_Gen_Model'

base_tokenizer = AutoTokenizer.from_pretrained(model_path)
base_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

## Model Eval (BLEU)

In [ ]:
!pip install evaluate

In [6]:
from evaluate import load
#Loading the BLEU score metric
bleu = load("bleu")

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained('Story_Gen_Model/Basic/')
tokenizer = AutoTokenizer.from_pretrained('t5-small') 

In [19]:
def encode_input(x, base_tokenizer = tokenizer):
  x = str(x)
  return base_tokenizer(x, max_length = 512, truncation=True, padding='max_length', return_tensors='pt').input_ids

In [20]:
df = pd.read_csv('Plot_Summary_Dataset')
train_df, test_df = train_test_split(df, test_size=0.1, random_state=1)
test_df['Input'] = test_df['Input'].apply(encode_input)

In [30]:
test_df.reset_index(drop=True, inplace=True)

In [25]:
num_beams = 1 
top_p = 1.0
do_sample = False
#Greedy, Ancestral and Nucleus
decoding_strats = [[1,1.0, False],[1,1.0, True],[1,0.9, True]]

In [ ]:
#BLEU score eval of FT model
results = []
for x in decoding_strats:
  predictions = []
  references = []
  num_beams, top_p, do_sample = x
  for idx,row in test_df.iterrows():
    references.append(test_df['Summary'].iloc[idx])
    predictions.append(tokenizer.batch_decode(model.generate(test_df['Input'].iloc[idx], num_beams = num_beams, top_p = top_p, do_sample = do_sample, min_length = len(test_df['Summary'].iloc[idx]), max_length = len(test_df['Summary'].iloc[idx])), skip_special_tokens=True))
  base_results = bleu.compute(predictions=predictions, references=references)
  results.append(base_results)
print(results)

## Loss curve

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Loading the trainer_state json file, which consists of the losses at different steps/epochs
df = pd.read_json('Add path to the json file')

In [ ]:
#Abstraction of the loss and epochs from the log history 
def get_epoch(x):
  return x['epoch']

def get_loss(x):
  return x['loss']

In [ ]:
#Organize in a way to extract losses and steps
df['epoch_number'] = df['log_history'].apply(get_epoch)
df ['loss'] = df['log_history'].apply(get_loss)

In [ ]:
#Plotting of the training loss curve for the FT Model 
plt.plot(df['epoch_number'].values, df['loss'].values) 
plt.xlabel('#Epochs')
plt.ylabel('Loss value')
plt.title('Training curve T5')